In [1]:
import numpy as np
import os

In [2]:
from smpl_np import SMPLModel
from reader import parse_amc, parse_asf
from imitator import Imitator

In [3]:
smpl_path = "../../models/smplpytorch/smplpytorch/native/models/basicmodel_neutral_lbs_10_207_0_v1.1.0.pkl"
smpl = SMPLModel(smpl_path)

In [4]:
# loop through cmu databases and raed ams asf
poses = []
root = "../../data/all_asfamc/subjects/"
for folder in os.scandir(root):
    # print(folder.path)
    # print(folder.name)
    subject_id = folder.name
    if folder.is_dir:
        asf_path = os.path.join(folder.path, f"{subject_id}.asf")
        if (os.path.exists(asf_path)):
            # print("yes")
            joints = parse_asf(asf_path)
            imit = Imitator(joints, smpl)
            # print(joints)
            for file in os.scandir(folder):
                # print(file.name)
                # read amc and asf
                if (file.name.endswith(".amc")):
                    motion = parse_amc(file.path)
                    print(len(motion))
                    theta = imit.motion2theta(motion[0])
                    poses.append(theta)
                    print(theta)
                    imit.imitate(motion[0])
                    print(smpl.verts.shape)
                    print(smpl.faces.shape)

                break
        else:
            print(f".asf file for subject {subject_id} does not exist!")
            print(f"path {asf_path} does not exist, skip folder")
    break

3723
[[-1.04239785e-01  3.05344123e+00 -2.00212615e-02]
 [-2.01321802e-01 -3.46333828e-01 -1.42055009e-01]
 [-1.71852132e-01  2.87289213e-01  2.76143906e-01]
 [-6.25348320e-02  3.95460216e-02  1.27162772e-02]
 [ 4.89237017e-01  6.77641379e-03  1.19182934e-01]
 [ 3.63706897e-01 -2.35706109e-03 -1.33946486e-01]
 [-1.05540225e-01  1.92421352e-02 -3.12519030e-02]
 [-5.08493324e-01  3.48774168e-01 -2.22792156e-01]
 [-2.98874072e-01 -3.30543367e-01  2.95657160e-01]
 [-1.50386384e-01 -4.25308514e-03 -7.01353147e-02]
 [-2.52225190e-01 -9.18024613e-02  2.15771193e-17]
 [ 5.54110150e-02 -2.01679601e-02 -3.64159416e-17]
 [ 3.97085944e-01  1.94127199e-02  6.62252301e-02]
 [-1.03480497e-16  6.14277173e-16  2.54360320e-16]
 [-1.03480497e-16  6.14277173e-16  2.54360320e-16]
 [ 1.47636256e-01  1.53854284e-02  3.56679394e-02]
 [ 5.46269125e-01 -2.72725477e-01 -1.55646381e+00]
 [ 9.41827312e-01  5.39362933e-01  1.45979978e+00]
 [ 3.67605525e-17 -6.56986848e-01  3.79311534e-01]
 [-1.58688510e-16  5.42788

In [5]:
smpl

In [9]:
import torch
import numpy as np

faces = torch.from_numpy(np.int64(smpl.faces))
verts = torch.FloatTensor(smpl.verts)

from pytorch3d.renderer import (
    FoVPerspectiveCameras, look_at_view_transform,
    RasterizationSettings, BlendParams,
    MeshRenderer, MeshRasterizer, HardPhongShader,
    TexturesVertex
)

from pytorch3d.vis.plotly_vis import AxisArgs, plot_batch_individually, plot_scene

from pytorch3d.structures import Meshes

# Initialize an OpenGL perspective camera.
R, T = look_at_view_transform(1, 0, 0)
cameras = FoVPerspectiveCameras(R=R, T=T)

# Define the settings for rasterization and shading. Here we set the output image to be of size
# 512x512. As we are rendering images for visualization purposes only we will set faces_per_pixel=1
# and blur_radius=0.0. Refer to rasterize_meshes.py for explanations of these parameters.
raster_settings = RasterizationSettings(
    image_size=512,
    blur_radius=0.0,
    faces_per_pixel=1,
)

# Create a Phong renderer by composing a rasterizer and a shader. Here we can use a predefined
# PhongShader, passing in the device on which to initialize the default parameters
renderer = MeshRenderer(
    rasterizer=MeshRasterizer(cameras=cameras, raster_settings=raster_settings),
    shader=HardPhongShader(cameras=cameras)
)

# color = torch.ones(1, verts.shape[0], 3) * 0.9
# textures = TexturesVertex(verts_features=color)
mesh = Meshes(verts=[verts], faces=[faces])

fig = plot_scene({
            "subplot1": {
                "cow_mesh": mesh,
                # "camera": cameras
            },
        }) #, viewpoint_cameras=self.cameras)
fig.show()